# Sample code for performing obstacle avoidance #

Import necessary packages.

In [1]:
# Code adapted from: https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/autonomousSequence.py

import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

# CrazyFlie imports:

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander

Set your group number and camera number.

In [4]:
group_number = 3

# we have about 92cm in both left and right direction, ADJUST THIS
min_y_pos = -0.7 # ADJUST THIS
max_y_pos = 0.7 # ADJUST THIS
max_x_pos = 3.5 # ADJUST THIS

# thresholds for determining whether obstacle is centered
center_threshold = 340
right_threshold_red = 450
left_threshold_red = 210

# thresholds for determinig whether book is centered
right_threshold_blue = 390
left_threshold_blue = 290

# thresholds for determining distance from objects
red_width_thr = 70 # ADJUST THIS
blue_width_thr = 50 # ADJUST THIS

# Possibly try 0, 1, 2 ...
camera = 1

## Helper functions ##

The following cell contains some sample functions which will be useful.

In particular, __check_contours__ and __findGreatesContour__ will perform red filtering on the live camera feed and identify the obstacles. The red filtering is controlled by setting HSV intervals in the __check_contours__ function. Note that the intervals will require tuning and may vary on different drones/cameras.

The __adjust_position__ function can also be modified for performing obstacle avoidance.

In [9]:
import sys

# Get the current crazyflie position:
def position_estimate(scf):
    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]
            x = data['kalman.varPX']
            y = data['kalman.varPY']
            z = data['kalman.varPZ']
            
    print(x, y, z)
    return x, y, z


# Set the built-in PID controller:
def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    time.sleep(2)
    return


# Ascend and hover at 1m:
def ascend_and_hover(cf):
    # Ascend:
    for y in range(5):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(0.1)
    # Hover at 0.75 meters:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)
    return


# Sort through contours in the image
def findGreatesContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1

    #print(largest_area)

    return largest_area, largest_contour_index


# Find contours in the image
def check_red_contours(frame):

    print('Checking image for red contours:')

    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)

    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.    
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    largest_area, largest_contour_index = findGreatesContour(contours)
    contour_x = -1
    width_x = -1
    
    if largest_contour_index != -1:
        largest_contour = contours[largest_contour_index]
        contour_x = np.mean(largest_contour, axis=0).flatten()[0]
        max_x = np.max(largest_contour, axis=0).flatten()[0]
        min_x = np.min(largest_contour, axis=0).flatten()[0]
        width_x = max_x - min_x
    
        #print(largest_area)
    
        if largest_area > 100:
            return mask, True, contour_x, width_x
            
    return mask, False, contour_x, width_x

def check_blue_contours(frame):

    print('Checking image for blue contours:')

    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (110, 50, 50)
    ub1 = (130, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask = cv2.inRange(hsv1, lb1, ub1)

    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.    
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    largest_area, largest_contour_index = findGreatesContour(contours)
    contour_x = -1
    width_x = -1
    
    if largest_contour_index != -1:
        largest_contour = contours[largest_contour_index]
        contour_x = np.mean(largest_contour, axis=0).flatten()[0]
        max_x = np.max(largest_contour, axis=0).flatten()[0]
        min_x = np.min(largest_contour, axis=0).flatten()[0]
        width_x = max_x - min_x
    
        #print(largest_area)
    
        if largest_area > 100:
            return mask, True, contour_x, width_x
            
    return mask, False, contour_x, width_x


# Follow the setpoint sequence trajectory:
def adjust_position(cf, current_x, current_y, direction):

    print('Adjusting position')

    steps_per_meter = int(10)
    # Set the number here (the iterations of the for-loop) to the number of side steps.
    # You may choose to tune the number and size of the steps.
    for i in range(1): 
        if direction == "RIGHT":
            current_y = current_y - 1.0/float(steps_per_meter)   
        elif direction == "FORWARD":
            current_x = current_x + 1.0/float(steps_per_meter)
        elif direction == "LEFT":
            current_y = current_y + 1.0/float(steps_per_meter)

        position = [current_x, current_y, 0.5, 0.0]
                        
        print('Setting position {}'.format(position))
        for i in range(10):
            
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            
            time.sleep(0.1)

    #cf.commander.send_stop_setpoint()
    # Make sure that the last packet leaves before the link is closed.
    # The message queue is not flushed before closing.
    #time.sleep(0.05)
    return current_x, current_y


# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    print('Descending')
    # Hover at 0.75 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 25)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return

## Tune the color filtering ##

You can use the following cell to test and visualize the red filtering. This cell *not* make the drone fly. It will connect to the CrazyFlie camera and perform red filtering on the live video feed. You should use this cell to tune the HSV intervals, and then copy/paste your tuned intervals into the __check_contours__ function below. When tuning the intervals, keep in mind that the lighting in the environment can matter.

### Red filtering ###

In [ ]:
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    #middle = frame[:, 165:315] # only looks at middle 1/3ish of x axis; takes entire y-axis
    
    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #hsv1 = cv2.cvtColor(middle, cv2.COLOR_BGR2HSV)
    #hsv2 = cv2.cvtColor(middle, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)
    
    # Compute
    cv2.imshow('mask', mask)    

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

### Blue filtering ###

In [9]:
""" TEST BLUE FILTERING """

cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (110, 50, 50)
    ub1 = (130, 255, 255)
    
    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #hsv1 = cv2.cvtColor(middle, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)

    contours, hierarchy = cv2.findContours(mask1, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    largest_area, largest_contour_index = findGreatesContour(contours)
    contour_x = -1
    width_x = -1
    
    if largest_contour_index != -1:
        largest_contour = contours[largest_contour_index]
        contour_x = np.mean(largest_contour, axis=0).flatten()[0]
        max_x = np.max(largest_contour, axis=0).flatten()[0]
        min_x = np.min(largest_contour, axis=0).flatten()[0]
        width_x = max_x - min_x
        print("contour centered at %.3f with width %.3f"%(contour_x, width_x))
    
    # Compute
    cv2.imshow('mask1', mask1)    

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    time.sleep(1)    

# Release the capture
cap.release()
cv2.destroyAllWindows()

[ WARN:0@542.698] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


contour centered at 312.240 with width 125.000
contour centered at 308.921 with width 152.000
contour centered at 312.450 with width 153.000
contour centered at 305.561 with width 153.000
contour centered at 307.652 with width 160.000
contour centered at 280.628 with width 159.000
contour centered at 282.656 with width 161.000
contour centered at 291.796 with width 155.000
contour centered at 299.101 with width 153.000
contour centered at 298.502 with width 154.000
contour centered at 387.697 with width 114.000
contour centered at 434.753 with width 78.000
contour centered at 422.662 with width 111.000
contour centered at 424.250 with width 112.000
contour centered at 436.923 with width 82.000
contour centered at 131.234 with width 148.000
contour centered at 113.596 with width 148.000
contour centered at 123.569 with width 147.000
contour centered at 134.037 with width 159.000
contour centered at 282.085 with width 125.000
contour centered at 306.114 with width 112.000


## Test obstacle avoidance on the CrazyFlie ##

The following cell *will* fly the drone. Place the CrazyFlie in front of an obstacle in the netted area for testing. This cell will perform object detection and avoidance using the red filtering defined in the helper functions above.

### Avoiding obstacles ###

In [5]:
import cv2
import time
import numpy as np

# ************ Parameters that might be useful to change ************ 
# Set the URI the Crazyflie will connect to
uri = f'radio://0/{group_number}/2M'
# ******************************************************************

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascend to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0
        direction = "RIGHT"

        # capture the video
        cap = cv2.VideoCapture(camera)

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False

        # Ascend and hover a bit
        set_PID_controller(cf)
        ascend_and_hover(cf)
        time.sleep(1)
        
        current_x = 0
        current_y = 0

        emptycounter = 0
        
        # detect objects in each frame of the video
        while cap.isOpened() and current_x < max_x_pos and time.time() - t < 70:
            
            # Try to read image
            ret, frame = cap.read()
            if ret:
                                
                # if drone is near boundaries, move towards center 
                if current_y < min_y_pos:
                    direction = "LEFT"
                    current_x, current_y = adjust_position(cf, current_x, current_y, direction)
                    current_x, current_y = adjust_position(cf, current_x, current_y, direction)
                    
                elif current_y > max_y_pos:
                    direction = "RIGHT"
                    current_x, current_y = adjust_position(cf, current_x, current_y, direction)
                    current_x, current_y = adjust_position(cf, current_x, current_y, direction)

                else:  
                    mask_red, contour_bool_red, contour_x_red, width_x_red = check_red_contours(frame)

                    # check for obstacles
                    if(contour_bool_red):
                        emptycounter = 0
                        print("theres a contour centered at %.3f with width %.3f"%(contour_x_red, width_x_red))
                        # if obstacle is not centered in image, move forward
                        if contour_x_red < left_threshold_red or contour_x_red > right_threshold_red or width_x_red < red_width_thr:
                            direction = "FORWARD"
                        elif contour_x_red < center_threshold:
                            direction = "RIGHT"
                        else:
                            direction = "LEFT"
                    else:
                        print("no obstacle detected")
                        emptycounter += 1
                        if emptycounter == 3:
                            break
                        direction = "FORWARD"

                    print("going %s" %(direction))
                    current_x, current_y = adjust_position(cf, current_x, current_y, direction)                    

                # Check image
                cv2.imshow('mask_red', mask_red)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
            else:
                print('no image!!')
                
        cap.release()
        
        # Descend and stop all motion:
        print('target reached!')
        hover_and_descend(cf)
        
        cv2.destroyAllWindows()

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/12/2M
radio://0/3/2M
radio://0/12/2M


[ WARN:0@98.460] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Initializing PID Controller
Checking image for red contours:
theres a contour centered at 317.896 with width 27.000
going FORWARD
Adjusting position
Setting position [0.1, 0, 0.5, 0.0]
Checking image for red contours:
theres a contour centered at 313.963 with width 57.000
going FORWARD
Adjusting position
Setting position [0.2, 0, 0.5, 0.0]
Checking image for red contours:
theres a contour centered at 291.076 with width 37.000
going FORWARD
Adjusting position
Setting position [0.30000000000000004, 0, 0.5, 0.0]
Checking image for red contours:
theres a contour centered at 264.171 with width 84.000
going RIGHT
Adjusting position
Setting position [0.30000000000000004, -0.1, 0.5, 0.0]
Checking image for red contours:
theres a contour centered at 252.866 with width 66.000
going FORWARD
Adjusting position
Setting position [0.4, -0.1, 0.5, 0.0]
Checking image for red contours:
theres a contour centered at 250.852 with width 60.000
going FORWARD
Adjusting position
Setting position [0.5, -0.1, 0

KeyboardInterrupt: 

### Entire logic including telling how far an obstacle is, finding the book, etc ###

In [10]:
""" walking through entire logic """
import cv2
import time
import numpy as np

# ************ Parameters that might be useful to change ************ 
# Set the URI the Crazyflie will connect to
uri = f'radio://0/{group_number}/2M'
# ******************************************************************

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascend to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0
        direction = "RIGHT"

        # capture the video
        cap = cv2.VideoCapture(camera)

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False

        # scan flag
        scan_right = True

        # Ascend and hover a bit
        set_PID_controller(cf)
        ascend_and_hover(cf)
        time.sleep(1)
        
        current_x = 0
        current_y = 0

        last_center = -1.0
        
        # detect objects in each frame of the video
        while cap.isOpened() and not exit_loop:
            
            # Try to read image
            ret, frame = cap.read()
            if ret:
    
                # if drone is near boundaries, move towards center 
                if current_y < min_y_pos:
                    direction = "LEFT"
                    current_x, current_y = adjust_position(cf, current_x, current_y, direction)
                    current_x, current_y = adjust_position(cf, current_x, current_y, direction)
                    
                elif current_y > max_y_pos:
                    direction = "RIGHT"
                    current_x, current_y = adjust_position(cf, current_x, current_y, direction)
                    current_x, current_y = adjust_position(cf, current_x, current_y, direction)
                
                else:  
                    mask_red, contour_bool_red, contour_x_red, width_x_red = check_red_contours(frame)

                    # check for obstacles
                    if contour_bool_red and current_x < max_x_pos:
                        print("theres a contour centered at %.3f with width %.3f"%(contour_x_red, width_x_red))
                        # if obstacle is not centered in image, move forward
                        if contour_x_red < left_threshold_red or contour_x_red > right_threshold_red or width_x_red < red_width_thr:
                            direction = "FORWARD"
                        elif contour_x_red < center_threshold:
                            direction = "RIGHT"
                        else:
                            direction = "LEFT"

                    # if no obstacle is visible, check for book
                    else:
                        if current_x < max_x_pos:
                            direction = "FORWARD"
                        else:
                            # if at end of obstacle field, scan for book
                            mask_blue, contour_bool_blue, contour_x_blue, width_x_blue = check_blue_contours(frame)
    
                            if(contour_bool_blue):
                                print("contour centered at %.3f with width %.3f"%(contour_x_blue, width_x_blue))
                                if contour_x_blue > left_threshold_blue and contour_x_blue < right_threshold_blue:
                                    if last_center == -1.0 or (last_center != -1.0 and abs(contour_x_blue - last_center) < 100):
                                        print("close enough! landing")
                                        # close enough!
                                        break
                                elif contour_x_blue < left_threshold_blue:
                                    direction = "LEFT"
                                else:
                                    direction = "RIGHT"
            
                                last_center = contour_x_blue
                            else:
                                # if no book is visible, scan
                                if current_y > min_y_pos and scan_right:
                                    direction = "RIGHT"
                                elif current_y < min_y_pos:
                                    direction = "LEFT"
                                    scan_right = False
                                elif current_y < max_y_pos and not scan_right:
                                    direction = "LEFT"
                                else:
                                    direction = "RIGHT"
                                    scan_right = True

                    print("going %s" %(direction))
                    current_x, current_y = adjust_position(cf, current_x, current_y, direction)                    

                # Check image
                cv2.imshow('mask_red', mask_red)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
            else:
                print('no image!!')
                
        cap.release()
        
        # Descend and stop all motion:
        print('target reached!')
        hover_and_descend(cf)
        
        cv2.destroyAllWindows()

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/3/2M
radio://0/3/2M
radio://0/3/2M
radio://0/3/2M
Initializing PID Controller
Checking image for red contours:
theres a contour centered at 480.164 with width 67.000
going FORWARD
Adjusting position
Setting position [0.1, 0, 0.5, 0.0]
Checking image for red contours:
theres a contour centered at 483.667 with width 74.000
going FORWARD
Adjusting position
Setting position [0.2, 0, 0.5, 0.0]
Checking image for red contours:
theres a contour centered at 534.532 with width 57.000
going FORWARD
Adjusting position
Setting position [0.30000000000000004, 0, 0.5, 0.0]
Checking image for red contours:
theres a contour centered at 559.490 with width 116.000
going FORWARD
Adjusting position
Setting position [0.4, 0, 0.5, 0.0]
Checking image for red contours:
theres a contour centered at 599.783 with width 111.000
going FORWARD
Adjusting position
Setting position [0.5, 0, 0.5, 0.0]
Checking image for red contours:
theres a contour cen

KeyboardInterrupt: 

### Simulation of entire logic (no crazyflie) ###

In [ ]:
"""
KNOWN ISSUES:
- oscillating left and right when two obstacles are too close to each other
- what to do when there's an obstacle super close to the edge (y-dir) of box
"""


%matplotlib inline
import random
book_x = 3
book_y = 0

## Helper functions
# Get obstacles
def get_obstacles():
    x_locs = np.zeros(5)
    y_locs = np.zeros(5)
    
    for i in range(5):
        y_locs[i] = random.random() - 0.5
        x_locs[i] = random.random() * 3

    y_locs = y_locs[np.argsort(x_locs)]
    x_locs = x_locs[np.argsort(x_locs)]

    return x_locs, y_locs

# Check for obstacles
def check_red_contours_sim(current_x, current_y, x_locs, y_locs):
    contour_bool = False
    contour_x = -1
    width_x = -1
    
    for i in range(5):
        if abs(y_locs[i] - current_y) < 0.3 and x_locs[i] > current_x:
            print("Obstacle %d detected"%(i))
            contour_bool = True
            contour_x = 330 + (current_y - y_locs[i]) * 550
            
            if x_locs[i] - current_x < 0.3:
                width_x = 500 

            break
        else:
            continue
            
    return contour_bool, contour_x, width_x

# Check for book
def check_blue_contours_sim(current_x, current_y):
    contour_bool = False
    contour_x = -1
    width_x = -1

    if abs(book_y - current_y) < 0.1:
        contour_bool = True
        contour_x = 330 + (current_y - y_locs[i]) * 550
            
        if book_x - current_x < 0.1:
            width_x = 500 

    return contour_bool, contour_x, width_x

# Adjust position
def adjust_position_sim(current_x, current_y, direction):
    print('Adjusting position')

    steps_per_meter = int(15)
    # Set the number here (the iterations of the for-loop) to the number of side steps.
    # You may choose to tune the number and size of the steps.
    for i in range(3): 
        calc_y = current_y
        if direction == "RIGHT":
            calc_y = current_y - 1.0/float(steps_per_meter)   
        elif direction == "FORWARD":
            current_x = current_x + 1.0/float(steps_per_meter)
        elif direction == "LEFT":
            calc_y = current_y + 1.0/float(steps_per_meter)

        if calc_y > max_y_pos or calc_y < min_y_pos:
            current_x += 1.0/float(steps_per_meter)
            position = [current_x, current_y, 0.5, 0.0]
                        
            print('Setting position {}'.format(position))
            break
        else:
            current_y = calc_y
        
        position = [current_x, current_y, 0.5, 0.0]
                        
        print('Setting position {}'.format(position))

    return current_x, current_y

def check_bonk(current_x, current_y, x_locs, y_locs):
    for i in range(5):
        x_dist = x_locs[i] - current_x
        y_dist = y_locs[i] - current_y
        if np.sqrt(x_dist ** 2 + y_dist ** 2) < 0.075:
            print("bonked into obstacle %d"%(i))
            return True

    return False

# Plot everything
def plot_sim(x_locs, y_locs, x_traj, y_traj):
    fig, ax = plt.subplots()
    
    ax.set_xlim([0, 3])
    ax.set_ylim([-0.5, 0.5])
    ax.set_aspect('equal')   
    
    # plot obstacles and book
    for i in range(5):
        ax.add_artist(plt.Circle(tuple(np.array([x_locs[i], y_locs[i]])), 0.05))
    
    # plot trajectory
    plt.plot(x_traj, y_traj, 'ko-')
    plt.show()

current_x = 0.0
current_y = 0.0
x_locs, y_locs = get_obstacles()
x_traj = []
y_traj = []

print("Locations of obstacles: ")
for i in range(5):
    print("%.2f, %.2f"%(x_locs[i], y_locs[i]))

while current_x < 3.1:
    print("\nCurrent Location: %.3f, %.3f"%(current_x, current_y))

    # if too close to end of zone, just head to middle
    if current_x > 2.8:
        current_y = 0
        break
                    
    # if drone is near boundaries, move towards center 
    if current_y < min_y_pos:
        print("too far right, going left")
        direction = "LEFT"
        current_x, current_y = adjust_position_sim(current_x, current_y, direction)
        current_x, current_y = adjust_position_sim(current_x, current_y, direction)
        
    elif current_y > max_y_pos:
        print("too far left, going right")
        direction = "RIGHT"
        current_x, current_y = adjust_position_sim(current_x, current_y, direction)
        current_x, current_y = adjust_position_sim(current_x, current_y, direction)

    else:  
        contour_bool_red, contour_x_red, width_x_red = check_red_contours_sim(current_x, current_y, x_locs, y_locs)

        # check for obstacles
        if(contour_bool_red):
            print("theres an obstacle centered at %.3f with width %d"%(contour_x_red, width_x_red))
            # if obstacle is not centered in image or is too far away, move forward
            if contour_x_red < left_threshold_red or contour_x_red > right_threshold_red or width_x_red < red_width_thr:
                direction = "FORWARD"
            elif contour_x_red < center_threshold:
                direction = "RIGHT"
            else:
                direction = "LEFT"

        # if no obstacle is visible, check for book
        else:
            contour_bool_blue, contour_x_blue, width_x_blue = check_blue_contours_sim(current_x, current_y)

            if(contour_bool_blue):
                print("theres a book")
                if contour_x_blue > left_threshold_blue or contour_x_blue < right_threshold_blue:
                    # close enough!
                    print("close enough!")
                    break
                elif contour_x_blue < left_threshold_blue:
                    direction = "RIGHT"
                else:
                    direction = "LEFT"
            else:
                print("no obstacle or book detected")
                # if no book is visible, go foward and then move towards center
                current_x, current_y = adjust_position_sim(current_x, current_y, "FORWARD")
                if current_y > 0:
                    direction = "RIGHT"
                else:
                    direction = "LEFT"

        print("going %s" %(direction))
        current_x, current_y = adjust_position_sim(current_x, current_y, direction)

    x_traj.append(current_x)
    y_traj.append(current_y)
    if check_bonk(current_x, current_y, x_locs, y_locs):
        break

plot_sim(x_locs, y_locs, x_traj, y_traj)

    

### Test adjust_position() ###

In [4]:
"""test adjust pos DOES NOT USE CAMERA STUFF AT ALL """

# Set the URI the Crazyflie will connect to
uri = f'radio://0/{group_number}/2M'

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

# Check that CrazyFlie devices are available:
if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascent to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf
        current_x = 0.0
        current_y = 0.0

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        direction = "RIGHT"

        while(elapsed < 120):

            elapsed = time.time() - t
            if(elapsed > 5.0):

                if(ascended_bool==0):
                    set_PID_controller(cf)
                    ascend_and_hover(cf)
                    ascended_bool = 1
                else: 
                    """
                    if current_y < min_y_pos:
                        direction = "LEFT"
                    elif current_y > max_y_pos:
                        direction = "RIGHT"
                    """
                    direction = "FORWARD"
                    current_x, current_y = adjust_position(cf, current_x, current_y, direction)
                    time.sleep(0.1)

        # Descend and stop all motion:
        hover_and_descend(cf)

print('Done!')

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/3/2M
radio://0/12/2M
Initializing PID Controller
Adjusting position
Setting position [0.1, 0.0, 0.5, 0.0]
Adjusting position
Setting position [0.2, 0.0, 0.5, 0.0]
Adjusting position
Setting position [0.30000000000000004, 0.0, 0.5, 0.0]
Adjusting position
Setting position [0.4, 0.0, 0.5, 0.0]
Adjusting position
Setting position [0.5, 0.0, 0.5, 0.0]
Adjusting position
Setting position [0.6, 0.0, 0.5, 0.0]
Adjusting position
Setting position [0.7, 0.0, 0.5, 0.0]
Adjusting position
Setting position [0.7999999999999999, 0.0, 0.5, 0.0]
Adjusting position
Setting position [0.8999999999999999, 0.0, 0.5, 0.0]
Adjusting position
Setting position [0.9999999999999999, 0.0, 0.5, 0.0]
Adjusting position
Setting position [1.0999999999999999, 0.0, 0.5, 0.0]
Adjusting position
Setting position [1.2, 0.0, 0.5, 0.0]
Adjusting position
Setting position [1.3, 0.0, 0.5, 0.0]
Adjusting position
Setting position [1.4000000000000001, 0.0, 0.5, 0

KeyboardInterrupt: 

### Test blue detection and landing ###

In [8]:
""" WORKING YAY """

import cv2
import time
import numpy as np

# ************ Parameters that might be useful to change ************ 
# Set the URI the Crazyflie will connect to
uri = f'radio://0/{group_number}/2M'
# ******************************************************************

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascend to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0
        direction = "RIGHT"

        # capture the video
        cap = cv2.VideoCapture(camera)

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False

         # Ascend and hover a bit
        set_PID_controller(cf)
        ascend_and_hover(cf)
        time.sleep(1)
        
        current_x = 0
        current_y = 0

        #while (time.time() - t < 2):
        #    time.sleep(0.1)

        last_center = -1.0
        
        # detect objects in each frame of the video
        while cap.isOpened() and not exit_loop:
            
            # Try to read image
            ret, frame = cap.read()
            if ret:
                                
                mask_blue, contour_bool_blue, contour_x_blue, width_x_blue = check_blue_contours(frame)

                if(contour_bool_blue):
                    print("contour centered at %.3f with width %.3f"%(contour_x_blue, width_x_blue))
                    if contour_x_blue > left_threshold_blue and contour_x_blue < right_threshold_blue:
                        if last_center == -1.0 or (last_center != -1.0 and abs(contour_x_blue - last_center) < 100):
                            print("close enough! landing")
                            # close enough!
                            break
                    elif contour_x_blue < left_threshold_blue:
                        direction = "LEFT"
                    else:
                        direction = "RIGHT"

                    last_center = contour_x_blue

                print("going %s" %(direction))
                current_x, current_y = adjust_position(cf, current_x, current_y, direction)                    

                # Check image
                #cv2.imshow('mask_blue', mask_blue)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
            else:
                print('no image!!')
                 
        cap.release()
        
        # Descend and stop all motion:
        print('target reached!')
        hover_and_descend(cf)
        
        cv2.destroyAllWindows()

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/12/2M
radio://0/14/2M
radio://0/3/2M
radio://0/12/2M
radio://0/14/2M
radio://0/3/2M
radio://0/12/2M
radio://0/14/2M
radio://0/12/2M
radio://0/14/2M
radio://0/3/2M
radio://0/12/2M
radio://0/14/2M


[ WARN:0@65658.295] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Initializing PID Controller
Checking image for blue contours:
contour centered at 608.556 with width 42.000
going RIGHT
Adjusting position
Setting position [0, -0.1, 0.5, 0.0]
Checking image for blue contours:
contour centered at 184.217 with width 73.000
going LEFT
Adjusting position
Setting position [0, 0.0, 0.5, 0.0]
Checking image for blue contours:
contour centered at 204.189 with width 70.000
going LEFT
Adjusting position
Setting position [0, 0.1, 0.5, 0.0]
Checking image for blue contours:
contour centered at 235.335 with width 80.000
going LEFT
Adjusting position
Setting position [0, 0.2, 0.5, 0.0]
Checking image for blue contours:
contour centered at 265.674 with width 84.000
going LEFT
Adjusting position
Setting position [0, 0.30000000000000004, 0.5, 0.0]
Checking image for blue contours:
contour centered at 303.849 with width 86.000
close enough! landing
target reached!
Descending:


## OUT OF DATE STUFF ##

In [ ]:
""" NOT MOST UP TO DATE """


# Set the URI the Crazyflie will connect to
uri = f'radio://0/{group_number}/2M'

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

# Check that CrazyFlie devices are available:
if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascent to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf
        current_x = 0.0
        current_y = 0.0

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0
        direction = "RIGHT"

        cap = cv2.VideoCapture(camera)
        #while(cap.isOpened()):
        #while(current_y < 30):
        while(cap.isOpened() and elapsed < 45):
            
            ret, frame = cap.read()
            elapsed = time.time() - t
            
            if(elapsed > 5.0):
                print("chk3")
                print('Capturing.....')

                if ret:
                    print("chk4")
                    frame = frame[:, 165:315] # only looks at middle 1/3ish of x axis; takes entire y-axis
                    print("chk5")
                    cv2.imshow('frame',frame)
                    print("chk6")
                    
                    if(ascended_bool==0):
                        set_PID_controller(cf)
                        ascend_and_hover(cf)
                        ascended_bool = 1
                    else:

                        if(check_red_contours(frame)):
                            print("theres a contour")
                            if current_y < min_y_pos:
                                direction = "LEFT"
                            elif current_y > max_y_pos:
                                direction = "RIGHT"   
                            
                        else:
                            direction = "FORWARD"

                        print("going %s" %(direction))
                        current_x, current_y = adjust_position(cf, current_x, current_y, direction)

                    
            

        cap.release()

        # Descend and stop all motion:
        hover_and_descend(cf)

print('Done!')

In [ ]:
""" original NOT UP TO DATE """

# Set the URI the Crazyflie will connect to
uri = f'radio://0/{group_number}/2M'

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

# Check that CrazyFlie devices are available:
if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascent to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf
        current_y = 0.0

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        cap = cv2.VideoCapture(camera)
        while(cap.isOpened()):

            ret, frame = cap.read()

            elapsed = time.time() - t
            if(elapsed > 5.0):

                print('Capturing.....')

                if ret:
                    cv2.imshow('frame',frame)

                    if(ascended_bool==0):
                        set_PID_controller(cf)
                        ascend_and_hover(cf)
                        ascended_bool = 1
                    else:

                        if(check_contours(frame)):
                            current_y = adjust_position(cf, current_y)

            if(elapsed > 10.0):
                        break

        cap.release()

        # Descend and stop all motion:
        hover_and_descend(cf)

print('Done!')